Esta partde do codigo só é nescessaria caso o projeto seja rodado no Google Colab

In [45]:
!pip install funcy
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Imports nescessarios

In [46]:
# import pickle
import os
import pandas as pd
import unidecode
import requests
import nltk
from gensim import corpora, models as gensim_models
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from pyLDAvis_local import *
from pyLDAvis_local import gensim_local


In [47]:
nltk.download('stopwords')
STOP_WORDS = stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Função responsavel por coletar informações como Titulo, Descrição e Link da noticia em diferentes categorias do G1

In [48]:
MAX_NOTICIAS = 5
def get_g1_topic_data(topic):
  URL = f'https://g1.globo.com/{topic}/'
  page = requests.get(URL)
  soup = BeautifulSoup(page.content, "html.parser")
  
  titles = soup.find_all('a', {'class' : 'feed-post-link gui-color-primary gui-color-hover'})
  descs = soup.find_all('div', {'class' : 'feed-post-body-resumo'})

  noticias = []
  for title, desc in zip(titles, descs):
    noticias.append({
      'title' : unidecode.unidecode(title.text), 
      'desc' : unidecode.unidecode(desc.text),
      'link' : title['href']
    })
    if len(noticias) == MAX_NOTICIAS:
      break
          
    
  df = pd.DataFrame(noticias)
  df['topic'] = topic
  return df

Agora para cada tema (categoria do G1) vamos pegar algumas noticias

In [50]:
topics = [
  'economia/agronegocios',
  'ciencia/',
  'carnaval/2022/',
  'df/distrito-federal/',
  'fato-ou-fake/',
  'economia/',
  'politica/eleicoes/2022/',
  'empreendedorismo/'
]
topicsDataFrames = [get_g1_topic_data(topic) for topic in topics]

Função responsavel por concatenar DataFrames

In [49]:
def concat_dataframes(dataframes):
    df = pd.concat(dataframes).reset_index()
    del df['index']
    
    return df

In [51]:
concat_df = concat_dataframes(topicsDataFrames)

In [52]:
concat_df

,title,desc,link,topic
0,Brasil testa trigo transgenico no Cerrado a me...,Plantio teste esta sendo feito pela Empresa Br...,https://g1.globo.com/economia/agronegocios/not...,economia/agronegocios
1,1o Encontro Brasileiro de Degustadores de Cafe...,Evento teve inicio nesta segunda-feira (6) e v...,https://g1.globo.com/ro/rondonia/rondonia-rura...,economia/agronegocios
2,Violencia no campo: agricultores e policiais f...,Iniciativas estao melhorando enfrentamento con...,https://g1.globo.com/economia/agronegocios/glo...,economia/agronegocios
3,Beiju colorido melhora renda de mulheres em co...,"Iguaria, cuja materia-prima e a mandioca, ganh...",https://g1.globo.com/economia/agronegocios/glo...,economia/agronegocios
4,Mosca-do-estabulo preocupa pecuaristas em Pern...,Inseto chega a picar animais ate 8 vezes por m...,https://g1.globo.com/economia/agronegocios/glo...,economia/agronegocios
5,Cientistas fazem mapa detalhado do fundo do ma...,Um novo mapa traca a forma do leito do oceano ...,https://g1.globo.com/meio-ambiente/noticia/202...,ciencia/
6,Covid-19: bairros vulneraveis registram mais m...,Levantamento foi feito entre os meses de marco...,https://g1.globo.com/mg/zona-da-mata/noticia/2...,ciencia/
7,Os beneficios de uma dieta a base de plantas n...,Todos os profissionais da area da saude e gest...,https://g1.globo.com/bemestar/blog/longevidade...,ciencia/
8,Foto 51: a fascinante historia por tras da cel...,"Tirada ha 70 anos, a foto que foi essencial pa...",https://g1.globo.com/ciencia/noticia/2022/06/0...,ciencia/
9,Como fazer uma pocao de amor cientificamente c...,A medica biomedica Sophie Ward se propos a rea...,https://g1.globo.com/ciencia/noticia/2022/06/0...,ciencia/


In [53]:
def stopwords_column(df, new_name, column):
    df[new_name] = df[column].apply(lambda x: ' '.join([word for word in x.split() if word not in (STOP_WORDS)]))

    return df

In [54]:
concat_df = stopwords_column(concat_df, 'title_stopwords', 'title')
concat_df = stopwords_column(concat_df, 'desc_stopwords', 'desc')

In [55]:
def make_cluster_graph(df, column, num_topics, num_words):
    texts = [text.split() for text in df[column]]
    text_dictionary = corpora.Dictionary(texts) 
    corpus = [text_dictionary.doc2bow(text) for text in texts]
    
    ldamodel = gensim_models.ldamodel.LdaModel(corpus, num_topics = num_topics, id2word=text_dictionary, passes=15)
    topics = ldamodel.print_topics(num_words=num_words)
    lda_display = gensim_local.prepare(ldamodel, corpus, text_dictionary, sort_topics=False)
    
    return display(lda_display)

In [56]:
test_title = make_cluster_graph(concat_df, 'title_stopwords', len(topics), 6)
test_title

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [57]:
test_desc = make_cluster_graph(concat_df, 'desc_stopwords', len(topics), 6)
test_desc

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for